In [14]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from utilsforecast.plotting import plot_series
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from utilsforecast.plotting import plot_series
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx

# Load your data
df = pd.read_csv('./data/Clean_training.csv')

# --- Data Preprocessing ---
df['unique_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
df = df.rename(columns={'Date': 'ds', 'Target': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['IsHoliday'] = df['IsHoliday'].astype(int)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df[markdown_cols] = df[markdown_cols].fillna(0)
df = df.sort_values(['unique_id', 'ds'])

# --- Model Training ---
# Use the single largest time series for this demonstration
largest_series_id = df['unique_id'].value_counts().idxmax()
df_subset = df[df['unique_id'] == largest_series_id].copy()

print(f"Training model on the largest series: {largest_series_id}")

# Define the N-BEATSx model configuration
horizon = 12
input_size = 2 * horizon
model = NBEATSx(h=horizon,
                input_size=input_size,
                max_steps=500,
                scaler_type='standard',
                learning_rate=0.005)

# Wrap the model in the NeuralForecast object
nf = NeuralForecast(models=[model], freq='W')

# Train the model
nf.fit(df=df_subset)

# --- Forecasting and Visualization ---
# Generate predictions
Y_hat_df = nf.predict()

# As you like, here is the plot of the forecast 📈
fig = plot_series(df_subset, Y_hat_df, max_insample_length=150)
plt.tight_layout()
fig.savefig('nbeatsx_forecast.png')

print("\nSuccess! Forecast plot saved as nbeatsx_forecast.png")

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training model on the largest series: 9_96



  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
900       Non-trainable params
2.4 M     Total params
9.789     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


Success! Forecast plot saved as nbeatsx_forecast.png


<Figure size 640x480 with 0 Axes>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from utilsforecast.plotting import plot_series
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx

# Load your data
df = pd.read_csv('./data/Clean_training.csv')

# --- Data Preprocessing ---
df['unique_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
df = df.rename(columns={'Date': 'ds', 'Target': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['IsHoliday'] = df['IsHoliday'].astype(int)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df[markdown_cols] = df[markdown_cols].fillna(0)
df = df.sort_values(['unique_id', 'ds'])

# --- Model Training ---
# Use the single largest time series for this demonstration
largest_series_id = df['unique_id'].value_counts().idxmax()
df_subset = df[df['unique_id'] == largest_series_id].copy()

print(f"Training model on the largest series: {largest_series_id}")

# Define the N-BEATSx model configuration
horizon = 12
input_size = 2 * horizon

# Split data into training and testing sets
train_df = df_subset[:-horizon]
test_df = df_subset

model = NBEATSx(h=horizon,
                input_size=input_size,
                max_steps=500,
                scaler_type='standard',
                learning_rate=0.005,
                random_seed=42) # Added for reproducibility

# Wrap the model in the NeuralForecast object
nf = NeuralForecast(models=[model], freq='W')

# Train the model on the training data
nf.fit(df=train_df)

# --- Forecasting and Visualization ---
# Generate predictions
Y_hat_df = nf.predict()

# As you like, here is the plot of the forecast against the test data 📈
fig = plot_series(test_df, Y_hat_df, max_insample_length=150, plot_random=False)
plt.tight_layout()
fig.savefig('nbeatsx_forecast_with_test.png')

print("\nSuccess! Forecast plot saved as nbeatsx_forecast_with_test.png")

Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
900       Non-trainable params
2.4 M     Total params
9.789     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Training model on the largest series: 9_96


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


Success! Forecast plot saved as nbeatsx_forecast_with_test.png


<Figure size 640x480 with 0 Axes>

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utilsforecast.plotting import plot_series
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Load your data
df = pd.read_csv('./data/Clean_training.csv')

# --- Data Preprocessing ---
df['unique_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
df = df.rename(columns={'Date': 'ds', 'Target': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['IsHoliday'] = df['IsHoliday'].astype(int)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df[markdown_cols] = df[markdown_cols].fillna(0)
df = df.sort_values(['unique_id', 'ds'])

# --- Model Training ---
largest_series_id = df['unique_id'].value_counts().idxmax()
df_subset = df[df['unique_id'] == largest_series_id].copy()

print(f"Training model on the largest series: {largest_series_id}")

horizon = 12
input_size = 2 * horizon

train_df = df_subset[:-horizon]
test_df = df_subset

model = NBEATSx(h=horizon,
                input_size=input_size,
                max_steps=500,
                scaler_type='standard',
                learning_rate=0.005,
                random_seed=42)

nf = NeuralForecast(models=[model], freq='W')
nf.fit(df=train_df)

# --- Forecasting ---
Y_hat_df = nf.predict()

# --- Visualization ---
fig = plot_series(test_df, Y_hat_df, max_insample_length=150, plot_random=False)
plt.tight_layout()
fig.savefig('nbeatsx_forecast_with_test.png')
print("\nSuccess! Forecast plot saved as nbeatsx_forecast_with_test.png")

# --- Model Evaluation ---
# Merge predictions with actual values from the test set
forecasts_df = pd.merge(test_df, Y_hat_df, on=['unique_id', 'ds'])

# Extract actual and predicted values
# --- Model Evaluation ---
# The true values are the last 'horizon' values from the test dataframe
y_true = test_df['y'].tail(horizon).values

# The predicted values are in the 'NBEATSx' column of the forecast output
y_pred = Y_hat_df['NBEATSx'].values

# Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)

print("\n--- Model Performance Metrics ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2%}")
print("---------------------------------")

Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
900       Non-trainable params
2.4 M     Total params
9.789     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Training model on the largest series: 9_96


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


Success! Forecast plot saved as nbeatsx_forecast_with_test.png

--- Model Performance Metrics ---
Mean Absolute Error (MAE): 262.14
Mean Squared Error (MSE): 155870.00
Mean Absolute Percentage Error (MAPE): 6.50%
---------------------------------


<Figure size 640x480 with 0 Axes>